Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff, LinearAlgebra

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
first(df, 5)

struct m_10_02d_model{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_02d_model)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack pr, β = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 10), β[1]) # a = X[1]
    ll += logpdf(Normal(0, 10), β[2]) # bp = X[2]
    ll += sum([loglikelihood(Binomial(1, logistic(dot(X[i, :], β))), [y[i]]) for i in 1:N])
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
X = hcat(ones(Int64, N), df[:prosoc_left]);
y = df[:pulled_left]
p = m_10_02d_model(y, X);
θ = (β = [1.0, 2.0], pr = ones(N),)
p(θ)

-487.6540051035728

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_02d_model) =
    as((β = as(Array, size(p.X, 2)), pr = as(Array, size(p.X, 2))))
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.083 s/step ...done
MCMC, adapting ϵ (25 steps)
0.088 s/step ...done
MCMC, adapting ϵ (50 steps)
0.096 s/step ...done
MCMC, adapting ϵ (100 steps)
step 100 (of 100), 0.1 s/step
0.1 s/step ...done
MCMC, adapting ϵ (200 steps)
step 100 (of 200), 0.098 s/step
step 200 (of 200), 0.099 s/step
0.099 s/step ...done
MCMC, adapting ϵ (400 steps)
step 100 (of 400), 0.096 s/step
step 200 (of 400), 0.1 s/step
step 300 (of 400), 0.11 s/step
step 400 (of 400), 0.11 s/step
0.11 s/step ...done
MCMC, adapting ϵ (50 steps)
0.099 s/step ...done
MCMC (1000 steps)
step 100 (of 1000), 0.12 s/step
step 200 (of 1000), 0.11 s/step
step 300 (of 1000), 0.11 s/step
step 400 (of 1000), 0.12 s/step
step 500 (of 1000), 0.12 s/step
step 600 (of 1000), 0.11 s/step
step 700 (of 1000), 0.11 s/step
step 800 (of 1000), 0.11 s/step
step 900 (of 1000), 0.1 s/step
step 1000 (of 1000), 0.1 s/step
0.1 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :pr),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [0.0884574, 0.572141], pr = [-10869.1, 28614.1])
 (β = [0.0584216, 0.597581], pr = [-10748.2, 28717.4])
 (β = [0.0107358, 0.547903], pr = [-10572.1, 28889.0])
 (β = [0.151348, 0.6593], pr = [-10570.1, 28885.8])   
 (β = [0.0740087, 0.659372], pr = [-10735.2, 28835.3])

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.054269518312904254
 0.5495693286291358  

Effective sample sizes (of untransformed draws)

In [10]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.94, min/25%/median/75%/max: 0.35 0.93 0.97 1.0 1.0
  termination: MaxDepth => 73% AdjacentTurn => 16% DoubledTurn => 11%
  depth: 1 => 0% 2 => 8% 3 => 9% 4 => 3% 5 => 4% 6 => 2% 7 => 0% 8 => 0% 10 => 73%


Result rethinking

In [11]:
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
a  0.04 0.12 -0.16  0.21   180 1.00
bp 0.57 0.19  0.30  0.87   183 1.01
";

Extract the parameter posterior means: `β`,

In [12]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.054269518312904254
 0.5495693286291358  

End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*